In [2]:
!pip install -q --upgrade transformers datasets peft bitsandbytes trl
!pip install -q accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.6 MB/s eta 0:00:00


In [3]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [4]:
import os
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    AutoConfig
)
from transformers import EarlyStoppingCallback 
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import login
from peft import LoraConfig, get_peft_model,PeftModel
from trl import SFTTrainer
import transformers
from torch.utils.data import DataLoader
from tqdm import tqdm

In [5]:
seed = 7
torch.manual_seed(seed)
np.random.seed(seed)

In [6]:
# token = "your_hugging_face_token"
login(token)

In [7]:
save_dir = ""
model_id = "meta-llama/Llama-3.2-3B-Instruct"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

In [9]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.2,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
len(tokenizer)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [12]:
# Define special tokens if needed
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"

special_tokens_dict = {}
if tokenizer.pad_token is None:
    special_tokens_dict['pad_token'] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict['eos_token'] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict['bos_token'] = DEFAULT_BOS_TOKEN

if special_tokens_dict:
    tokenizer.add_special_tokens(special_tokens_dict)

In [15]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=token
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [16]:
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, lora_config)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [17]:
dataset = load_dataset("Maxwell-Jia/MATH", trust_remote_code=True)

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.98M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [18]:
def preprocess_function(examples):
    inputs = [
        f"<|begin_of_text|><|start_header_id|>system <|end_header_id|>"
        "You are an expert math assistant<|eot_id|><|start_header_id|>user <|end_header_id|>"
        f"Solve the following math problem: {problem}\n"
        "Show all intermediate steps and please mandatorily include the final answer in LaTeX format in a box like \\boxed{{}}."
        "<|eot_id|><|start_header_id|> assistant <|end_header_id|>"
        for problem in examples['problem']
    ]
    targets = [
        f"{solution}{tokenizer.eos_token}"
        for solution in examples['solution']
    ]
    full_texts = [inp + tgt for inp, tgt in zip(inputs, targets)]
    model_inputs = tokenizer(
        full_texts,
        max_length=512,
        truncation=True,
        padding="longest",
        return_tensors="pt"
    )
    labels = model_inputs["input_ids"].clone()

    for i in range(len(labels)):
        input_ids = tokenizer(inputs[i], add_special_tokens=False).input_ids
        input_len = len(input_ids)
        labels[i][:input_len] = -100  # Mask the input tokens
    model_inputs["labels"] = labels
    return model_inputs

In [19]:
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [20]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False, 
    pad_to_multiple_of=8
)

In [21]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    warmup_ratio=0.1,
    num_train_epochs=5,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=100,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
    load_best_model_at_end=True,  # Load the best model at the end
    metric_for_best_model="eval_loss",  # Use eval_loss to select the best model
    greater_is_better=False,
    run_name="llama_finetuning_lora32",
    ddp_find_unused_parameters=False,
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,  # Stop training if no improvement after 2 evaluations
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Truncating train dataset:   0%|          | 0/7500 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
print("Training Started:")
trainer.train()

Training Started:


Step,Training Loss,Validation Loss
500,0.684200,0.687968
1000,0.637300,0.671475
1500,0.656100,0.654415
2000,0.473500,0.662007
2500,0.490700,0.661365


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=2500, training_loss=0.6351064056396485, metrics={'train_runtime': 12781.0135, 'train_samples_per_second': 2.934, 'train_steps_per_second': 0.734, 'total_flos': 9.854266638336e+16, 'train_loss': 0.6351064056396485})

In [24]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Push the model to the Hugging Face Hub
model.push_to_hub("baseline_mwp_sft_llama3.23b", use_auth_token=token)
tokenizer.push_to_hub("exp1_rank256_mwp_sft_llama3.23b", use_auth_token=token)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:922: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GT1999/exp1_rank256_mwp_sft_llama3.23b/commit/dcb0fb16a7126dfd9797ac5dd3fe06098048f8a0', commit_message='Upload tokenizer', commit_description='', oid='dcb0fb16a7126dfd9797ac5dd3fe06098048f8a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/GT1999/exp1_rank256_mwp_sft_llama3.23b', endpoint='https://huggingface.co', repo_type='model', repo_id='GT1999/exp1_rank256_mwp_sft_llama3.23b'), pr_revision=None, pr_num=None)

In [25]:
print(trainer.state.log_history)

[{'loss': 1.2514, 'grad_norm': 1.7609574794769287, 'learning_rate': 5.330490405117271e-06, 'num_tokens': 122122.0, 'mean_token_accuracy': 0.7339801675081253, 'epoch': 0.05333333333333334, 'step': 100}, {'loss': 0.7635, 'grad_norm': 1.4108306169509888, 'learning_rate': 1.0660980810234541e-05, 'num_tokens': 242348.0, 'mean_token_accuracy': 0.8015364521741867, 'epoch': 0.10666666666666667, 'step': 200}, {'loss': 0.6998, 'grad_norm': 1.2639617919921875, 'learning_rate': 1.5991471215351813e-05, 'num_tokens': 360171.0, 'mean_token_accuracy': 0.8068312817811966, 'epoch': 0.16, 'step': 300}, {'loss': 0.6543, 'grad_norm': 1.5057399272918701, 'learning_rate': 2.1321961620469083e-05, 'num_tokens': 476350.0, 'mean_token_accuracy': 0.8188773322105408, 'epoch': 0.21333333333333335, 'step': 400}, {'loss': 0.6842, 'grad_norm': 1.1411688327789307, 'learning_rate': 2.6652452025586356e-05, 'epoch': 0.26666666666666666, 'step': 500}, {'eval_loss': 0.6879682540893555, 'eval_runtime': 1380.8936, 'eval_sampl

In [26]:
model.eval()  # Set model to evaluation mode

test_samples = []
for idx in range(len(dataset['test'])):
    sample = dataset['test'][idx]
    input_text = (
        f"<|begin_of_text|><|start_header_id|>system <|end_header_id|>"
        "You are an expert math assistant<|eot_id|><|start_header_id|>user <|end_header_id|>"
        f"Solve the following math problem: {sample['problem']}\n"
        "Show all intermediate steps and please mandatorily include the final answer in LaTeX format in a box like \\boxed{{}}."
        "<|eot_id|><|start_header_id|> assistant <|end_header_id|>"
    )
    test_samples.append({
        "input_text": input_text,
        "problem": sample['problem'],
        "level": sample['level'],
        "type": sample['type'],
        "ground_truth": sample['solution']
    })
    
def collate_fn(batch):
    input_texts = [sample['input_text'] for sample in batch]
    model_inputs = tokenizer(
        input_texts,
        padding=True,
        truncation=True,
        max_length=1024,  
        return_tensors="pt"
    )
    model_inputs = {k: v.to(model.device) for k, v in model_inputs.items()}
    return model_inputs, batch

batch_size = 32 
test_dataloader = DataLoader(test_samples, batch_size=batch_size, collate_fn=collate_fn)

results_list = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for batch_idx, (model_inputs, batch_samples) in enumerate(tqdm(test_dataloader, desc=f"Evaluating")):
    current_batch_size = model_inputs['input_ids'].size(0)
    # Generate predictions
    try:
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=model_inputs['input_ids'],
                attention_mask=model_inputs['attention_mask'],
                max_new_tokens=512,  
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id
            )
        # Decode the outputs
        for i in range(current_batch_size):
            predicted_text = tokenizer.decode(output_ids[i], skip_special_tokens=True)
            # Store the results
            results_list.append({
                "problem": batch_samples[i]['problem'],
                "level": batch_samples[i]['level'],
                "type": batch_samples[i]['type'],
                "ground_truth": batch_samples[i]['ground_truth'],
                "predicted_solution": predicted_text
            })
    except Exception as e:
        print(f"Error during generation at batch {batch_idx+1}: {e}")
        for i in range(current_batch_size):
            results_list.append({
                "problem": batch_samples[i]['problem'],
                "level": batch_samples[i]['level'],
                "type": batch_samples[i]['type'],
                "ground_truth": batch_samples[i]['ground_truth'],
                "predicted_solution": ""  # Empty string for predicted_text
            })
        continue  


    if (batch_idx + 1) % 100 == 0:
        results_df = pd.DataFrame(results_list)
        results_save_path = os.path.join(save_dir, f"test_results_batch_{batch_idx+1}.csv")
        results_df.to_csv(results_save_path, index=False)
        print(f"Saved test results up to batch {batch_idx+1} to {results_save_path}")


results_df = pd.DataFrame(results_list)
results_save_path = os.path.join(save_dir, f"test_results_overall.csv")
results_df.to_csv(results_save_path, index=False)
print(f"Saved test results to {results_save_path}")

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Evaluating:  64%|██████▎   | 100/157 [3:50:02<1:57:30, 123.69s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results,

Saved test results up to batch 100 to /kaggle/working/test_results_batch_100.csv


Evaluating: 100%|██████████| 157/157 [5:52:33<00:00, 134.74s/it]

Saved test results to /kaggle/working/test_results_overall.csv
